In [21]:
REGIONS = """
 אילת
 אילת
 אילת
 אילת
 אילת
 אילת
 אשדוד
 אשדוד
 אשדוד
 אשדוד
 אשדוד
 אשדוד
 אשקלון
 אשקלון
 אשקלון
 אשקלון
 אשקלון
 אשקלון
 באר שבע
 באר שבע
 באר שבע
 באר שבע
 באר שבע
 באר שבע
 באר שבע
 באר שבע
 באר שבע
 בית שאן
 בית שאן
 בית שאן
 בית שאן
 בית שאן
 בית שאן
 בית שמש
 בית שמש
 בית שמש
 בית שמש
 בית שמש
 בית שמש
 בת ים
 בת ים
 בת ים
 בת ים
 בת ים
 בת ים
 דימונה
 דימונה
 דימונה
 דימונה
 דימונה
 דימונה
 הרצליה
 הרצליה
 הרצליה
 הרצליה
 הרצליה
 הרצליה
 חדרה
 חדרה
 חדרה
 חדרה
 חדרה
 חדרה
 חיפה
 חיפה
 חיפה
 חיפה
 חיפה
 חיפה
 חיפה
 חיפה
 חיפה
 טבריה
 טבריה
 טבריה
 טבריה
 טבריה
 טבריה
 ירושלים
 ירושלים
 ירושלים
 ירושלים
 ירושלים
 ירושלים
 ירושלים
 ירושלים
 ירושלים
 כפר סבא
 כפר סבא
 כפר סבא
 כפר סבא
 כפר סבא
 כפר סבא
 מסעדה
 מסעדה
 מסעדה
 מסעדה
 מסעדה
 מרכז
 מרכז
 מרכז
 נהריה
 נהריה
 נהריה
 נהריה
 נהריה
 נהריה
 נצרת
 נצרת
 נצרת
 נצרת
 נצרת
 נצרת
 נצרת
 נצרת
 נתניה
 נתניה
 נתניה
 נתניה
 נתניה
 נתניה
 עכו
 עכו
 עכו
 עכו
 עכו
 עכו
 עפולה
 עפולה
 עפולה
 עפולה
 עפולה
 עפולה
 פתח תקוה
 פתח תקוה
 פתח תקוה
 פתח תקוה
 פתח תקוה
 פתח תקוה
 צפון
 צפת
 צפת
 צפת
 צפת
 צפת
 צפת
 קצרין
 קצרין
 קצרין
 קצרין
 קצרין
 קצרין
 קריות
 קריות
 קריות
 קריות
 קריות
 קריות
 קרית גת
 קרית גת
 קרית גת
 קרית גת
 קרית גת
 קרית גת
 קרית שמונה
 קרית שמונה
 קרית שמונה
 קרית שמונה
 קרית שמונה
 קרית שמונה
 ראשון לציון
 ראשון לציון
 ראשון לציון
 ראשון לציון
 ראשון לציון
 ראשון לציון
 רחובות
 רחובות
 רחובות
 רחובות
 רחובות
 רחובות
 רמלה
 רמלה
 רמלה
 רמלה
 רמלה
 רמלה
 תל אביב
 תל אביב
 תל אביב
 תל אביב
 תל אביב
 תל אביב
 תל אביב
 תל אביב
 תל אביב
מחוז הדרום
מחוז הצפון
מחוז דרום
דרום
צפון
מרכז
"""

TRIBUNAL_SEARCH_TEXTS = [
                ['האזורי לעבודה'],
                ['הארצי לעבודה'],
                ['המרכז להכשרה והשתלמות שופטים'],
                ['הנהלת בתי המשפט'],
                ['כלכלי במחוזי'],
                ['מיסים במחוזי'],
                ['משפחה בשלום', 'משפחה', 'ענייני משפחה', 'ענניני משפחה'],
                ['נוער בשלום', 'נוער'],
                ['ענינים מקומים בשלום', 'ענינים מקומים'],
                ['קהילתי בשלום', 'קהילתי'],
                ['תעבורה במחוזי'],
                ['תעבורה בשלום'],
                ['השלום'],
                ['המחוזי'],
                ['העליון'],
                ['מסעדה']
            ]

JUDGE_RANK_SENIOR = """
בכירה
בכיר
"""

JUDGE_RANK_RASHAM = """
רשמת
רשם
"""

JUDGE_RANK_DEPUTY = """
סגן
סגנית
"""

JUDGE_RANK_PRESIDENT = """
נשיאה
נשיא
"""

In [30]:
import yaml
from dataflows import Flow, checkpoint, load, printer, add_field, update_resource, dump_to_path
from datapackage_pipelines_migdar.flows.judges import judges_flow
from collections import defaultdict

stats = defaultdict(int)

def parse_years(rows):
    if rows.res.name == 'judge_events':
        for row in rows:
            stats['total'] += 1
            if row['event_datestring']:
                for year in range(1800,2100):
                    if str(year) in row['event_datestring']:
                        stats['year-{}'.format(year)] += 1
                        row['year'] = year
                        break
            if not row.get('year'):
                stats['no-year'] += 1
                row['year'] = None
            yield row
    else:
        yield from rows

def parse_judge_role_appointments(text):
    tribunal_type, tribunal_region, judge_ranks = None, None, []
    for tribunal_search_texts in TRIBUNAL_SEARCH_TEXTS:
        type_desc = tribunal_search_texts[0]
        for search_text in tribunal_search_texts:
            if search_text.startswith('ה'):
                search_text = search_text[1:]
            if search_text in text:
                tribunal_type = type_desc
                break
        if tribunal_type:
            break
    for region in [r.strip() for r in REGIONS.splitlines() if r.strip() != '']:
        if region in text:
            tribunal_region = region
            break
    for ranks in [JUDGE_RANK_DEPUTY, JUDGE_RANK_PRESIDENT, JUDGE_RANK_RASHAM, JUDGE_RANK_SENIOR]:
        for rank in [r.strip() for r in ranks.splitlines() if r.strip() != '']:
            if rank in text:
                if rank not in judge_ranks:
                    judge_ranks.append(rank)
    return tribunal_type, tribunal_region, judge_ranks

judges_latest_event = {}
    
def parse_appointments(rows):
    if rows.res.name == 'judges_list':
        for row in rows:
            if row['Is_In_Dimus_List']:
                event_id = 'dimus_appointment'
            else:
                event_id = 'current_appointment'
            text = '{} {}'.format(row['Role_Description_1'], row['Role_Description_2'])
            judges_latest_event[row['Judge_ID']] = event = {}
            event.update(
                Judge_ID=row['Judge_ID'],
                First_Name=row['First_Name'],
                Last_Name=row['Last_Name'],
                event_id=event_id,
                event_datestring='',
                event_description=text,
                year=None
            )
            event['tribunal_type'], event['tribunal_region'], event['judge_ranks'] = parse_judge_role_appointments(text)
            yield row
    if rows.res.name == 'judge_events':
        last_judge_id = None
        last_appointment_year = None
        for row in rows:
            if last_judge_id is not None and last_judge_id != row['Judge_ID']:
                yield judges_latest_event[last_judge_id]
            # if row['event_id'] != 'appointment': continue
            text = '{} {}'.format(row['event_datestring'], row['event_description'])
            row['tribunal_type'],row['tribunal_region'], row['judge_ranks'] = parse_judge_role_appointments(text)
            yield row
            last_judge_id = row['Judge_ID']
        if last_judge_id:
            yield judges_latest_event[last_judge_id]
    else:
        yield from rows

def parse_appointment_year_ranges(rows):
    if rows.res.name == 'judge_events':
        judge_events = list(rows)
        last_appointment = None
        for event in judge_events:
            if event['event_id'] in ['appointment', 'retirement', 'death']:
                if last_appointment and last_appointment['Judge_ID'] == event['Judge_ID']:
                    last_appointment['end_year'] = event['year']
                last_appointment = event
            else:
                if event['event_id'] == ['current_appointment']:
                    if last_appointment and last_appointment['Judge_ID'] == event['Judge_ID']:
                        last_appointment['end_year'] = 0
                last_appointment = None
        for event in judge_events:
            if event['event_id'] == 'appointment':
                yield event
    else:
        yield from rows
        
Flow(
    # judges_flow('data/judges'),
    load('data/judges/datapackage.json'),
    add_field('year', 'integer', resources=['judge_events']),
    parse_years,
    add_field('tribunal_region', 'string', resources=['judge_events']),
    add_field('tribunal_type', 'string', resources=['judge_events']),
    add_field('judge_ranks', 'array', resource=['judge_events']),
    parse_appointments,
    add_field('end_year', 'year', resources=['judge_events']),
    parse_appointment_year_ranges,
    printer(tablefmt='html', num_rows=1, resources=[
        # 'judges_list', 
        'judge_events',
    ]),
    dump_to_path('data/judges_parsed'),
).process()

print(yaml.dump(dict(stats), default_flow_style=False))

#,Judge_ID (string),First_Name (string),Last_Name (string),event_id (string),event_datestring (string),event_description (string),year (integer),tribunal_region (string),tribunal_type (string),judge_ranks (array),end_year (year)
1,938984ba-645e-e811-8105-0050568a6817,מאיה,אב-גנים ויינשטיין,appointment,ביום 18/04/2010,מונתה לכהונת רשמת בית משפט השלום ירושלים.,2010,ירושלים,השלום,['רשמת'],2012
2,938984ba-645e-e811-8105-0050568a6817,מאיה,אב-גנים ויינשטיין,appointment,ביום 31/05/2012,מונתה לכהונת רשמת בכירה.,2012,,,"['רשמת', 'בכירה', 'בכיר']",2015
...,,,,,,,,,,,
4020,258684ba-645e-e811-8105-0050568a6817,מיכאל,תמיר,appointment,בינואר 2019,מונה לכהונת שופט בבית המשפט המחוזי מרכז-לוד,2019,מרכז,המחוזי,[],


no-year: 391
total: 17632
year-1900: 3
year-1901: 2
year-1902: 3
year-1903: 4
year-1904: 10
year-1905: 7
year-1906: 4
year-1907: 4
year-1908: 9
year-1909: 8
year-1910: 16
year-1911: 13
year-1912: 11
year-1913: 9
year-1914: 14
year-1915: 9
year-1916: 10
year-1917: 6
year-1918: 8
year-1919: 13
year-1920: 17
year-1921: 12
year-1922: 11
year-1923: 23
year-1924: 21
year-1925: 26
year-1926: 25
year-1927: 26
year-1928: 30
year-1929: 36
year-1930: 30
year-1931: 47
year-1932: 40
year-1933: 52
year-1934: 56
year-1935: 69
year-1936: 58
year-1937: 43
year-1938: 44
year-1939: 54
year-1940: 47
year-1941: 52
year-1942: 45
year-1943: 37
year-1944: 45
year-1945: 67
year-1946: 50
year-1947: 80
year-1948: 170
year-1949: 117
year-1950: 113
year-1951: 101
year-1952: 87
year-1953: 102
year-1954: 122
year-1955: 98
year-1956: 83
year-1957: 111
year-1958: 138
year-1959: 110
year-1960: 128
year-1961: 124
year-1962: 138
year-1963: 126
year-1964: 141
year-1965: 171
year-1966: 143
year-1967: 156
year-1968: 191
yea